In [ ]:
import requests
import pandas as pd
import re
import numpy as np

In [ ]:
# def get_data(url):
#     request = requests.get(url)
#     json_response = request.json()
#     return json_response

In [ ]:
# query="seo" #Define Your Query
# url = f"https://api.pushshift.io/reddit/search/comment/?q={query}"
# seo = get_data(url)

In [ ]:
!pip install psaw
from psaw import PushshiftAPI
api = PushshiftAPI()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Helper Functions
def removed(df, type): #return the percentage of removed submissions or comments
  
  total_rows = len(df)
  if type == 'sub':
    removed_rows = len(df.loc[df['selftext'] == '[removed]']) + len(df.loc[df['selftext'] == '[deleted]'])
    removed_precentage = (1- ((total_rows - removed_rows)/total_rows)) * 100
    print("submissions containing [removed] or [deleted] in selftext: ", round(removed_precentage, 3))
  elif type == 'com':
    removed_rows = len(df.loc[df['body'] == '[removed]']) + len(df.loc[df['body'] == '[deleted]'])
    removed_precentage = (1- ((total_rows - removed_rows)/total_rows)) * 100
    print("comments containing [removed] or [deleted] in body: ", round(removed_precentage, 3))
  else:
    removed_rows = len(df.loc[df['body'] == '[removed]']) + len(df.loc[df['body'] == '[deleted]']) + len(df.loc[df['selftext'] == '[removed]']) + len(df.loc[df['selftext'] == '[deleted]'])
    removed_precentage = (1- ((total_rows - removed_rows)/total_rows)) * 100
    print("comments/submissions containing [removed] or [deleted] in body: ", round(removed_precentage, 3))

In [ ]:
############################## GET SUBMISSIONS ##############################
def getSubmissions(topics, limit, num_comments):
  submissions_df1 = pd.DataFrame()
  for i in topics:
    gen = api.search_submissions(subreddit=i, # one or more subreddits to include in the search
                                filter=['subreddit','id','url','author', 'title', 'score',
                                          'subreddit','selftext','num_comments'], # list of fields to return
                                limit = limit, # limit on the number of records returned
                                num_comments=num_comments,
                                ) 
    submission_df = pd.DataFrame(list(gen))
    submissions_df1 = pd.concat([submissions_df1,submission_df])

  return submissions_df1

In [ ]:
#Get a minimum of 100 submissions from each of these subreddits:
topics1 = ['ExplainLikeImFive', 'FoodForThought','ChangeMyView', 'TodayILearned']
submissions_df1 = getSubmissions(topics1, 150, ">20")

/usr/local/lib/python3.7/dist-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


In [ ]:
# total rows
print("Total rows: ", len(submissions_df1))

Total rows:  600


In [ ]:
# submissions containing [removed] or [deleted] in selftext
removed(submissions_df1, 'sub')

submissions containing [removed] or [deleted] in selftext:  2.5


In [ ]:
# Get a minimum of 200 submissions for each of these topics
topics2 = ['AmericanPolitics', 'worldpolitics', 'government','tentree', 'nature', 'gardening','technology',
          'tech', 'gaming','bodybeast','briskwalking','crossfit','education', 'teaching', 'UniversityofReddit']

submissions_df2 = getSubmissions(topics2, 250, ">20")


In [ ]:
# total rows
print("Total rows: ", len(submissions_df2))

Total rows:  2780


In [ ]:
# submissions containing [removed] or [deleted] in selftext
removed(submissions_df2, 'sub')

submissions containing [removed] or [deleted] in selftext:  0.504


In [ ]:
# Combining everything
final_submissions = pd.DataFrame()
final_submissions = pd.concat([final_submissions,submissions_df1])
final_submissions = pd.concat([final_submissions,submissions_df2])
print("Final total rows: ", len(final_submissions))
removed(final_submissions, 'sub')
final_submissions = final_submissions.rename(columns={"url": "full_link"})

Final total rows:  3380
submissions containing [removed] or [deleted] in selftext:  0.858


In [ ]:
final_submissions.dtypes

author          object
created_utc      int64
id              object
num_comments     int64
score            int64
selftext        object
subreddit       object
title           object
full_link       object
created         object
d_              object
dtype: object

In [ ]:
# Add additional columns
topicsMap = {'explainlikeimfive': 'ExplainLikeImFive', 
             'Foodforthought': 'FoodForThought',
             'changemyview': 'ChangeMyView', 
             'todayilearned': 'TodayILearned', 
             'AmericanPolitics' : 'Politics', 
             'worldpolitics' : 'Politics', 
             'government' : 'Politics', 
             'republicans' : 'Politics', 
             'tentree' : 'Environment', 
             'nature' : 'Environment',
             'ZeroWaste' : 'Environment',
             'gardening' : 'Environment', 
             'technology' :'Technology', 
             'tech' :'Technology',
             'gaming' :'Technology', 
             'truegaming' :'Technology', 
             'bodybeast' : 'Healthcare', 
             'BodyBeast': 'Healthcare', 
             'briskwalking' : 'Healthcare', 
             'crossfit' : 'Healthcare', 
             'education' :'Education',
             'teaching' :'Education',
             'UniversityofReddit' :'Education'}

final_submissions['topic'] = 'blank'
final_submissions['created_at'] = final_submissions['created']

for i in range(len(final_submissions)):
  if final_submissions['subreddit'].iloc[i] in topicsMap:
    final_submissions['topic'].iloc[i] = topicsMap[final_submissions['subreddit'].iloc[i]]
  else:
    final_submissions['topic'].iloc[i] = final_submissions['subreddit'].iloc[i]

  date = final_submissions['created_utc'].iloc[i]
  final_submissions['created_at'].iloc[i] = pd.to_datetime(date, unit='s')

final_submissions['is_submission'] = True

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
finalSubmissions = final_submissions[['id', 'subreddit', 'full_link', 'title', 'selftext', 'author', 'is_submission', 'topic', 'created_at']]

In [ ]:
finalSubmissions[['id', 'subreddit', 'full_link', 'title','author', 'topic' ]] = finalSubmissions[['id', 'subreddit', 'full_link', 'title','author', 'topic']].astype('string')
finalSubmissions[['is_submission']] = finalSubmissions[['is_submission']].astype('boolean')
# finalSubmissions[['selftext']] = finalSubmissions[['selftext']].astype('text_en')
finalSubmissions[['created_at']] = finalSubmissions[['created_at']].astype('datetime64')

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:
finalSubmissions.dtypes

id                       string
subreddit                string
full_link                string
title                    string
selftext                 object
author                   string
is_submission           boolean
topic                    string
created_at       datetime64[ns]
dtype: object

In [ ]:
# Unique Topics
finalSubmissions['topic'].unique()

<StringArray>
[                                                                                                                                                                                                                                                                        'ExplainLikeImFive',
                                                                                                                                                                                                                                                                            'FoodForThought',
                                                                                                                                                                                                                                                                              'ChangeMyView',
                                                                                                                                

In [ ]:
finalSubmissions.to_pickle("finalSubmissions.pkl")

In [ ]:
finalSubmissions.to_csv("finalSubmissions.csv")

In [ ]:
finalSubmissions.groupby(['topic']).count()

,id,subreddit,full_link,title,selftext,author,is_submission,created_at
topic,,,,,,,,
ChangeMyView,150,150,150,150,150,150,150,150
Education,750,750,750,750,750,750,750,750
Environment,498,498,498,498,498,498,498,498
ExplainLikeImFive,150,150,150,150,150,150,150,150
FoodForThought,150,150,150,150,150,150,150,150
Healthcare,257,257,257,257,257,257,257,257
Politics,525,525,525,525,525,525,525,525
Technology,749,749,749,749,749,749,749,749
TodayILearned,150,150,150,150,150,150,150,150


In [ ]:
############################## GET COMMENTS ##############################
finalSubmissions = finalSubmissions.reset_index()
subreddit_details = finalSubmissions[['id', 'subreddit']].loc[finalSubmissions['selftext'] != '[removed]'].loc[finalSubmissions
                                                                                                               ['selftext'] != '[deleted]'].drop_duplicates(["id","subreddit"])
limit = round(30000/len(subreddit_details))
comments_df = pd.DataFrame()
for i in range(len(subreddit_details)):
  try:
    gen = api.search_comments(link_id=subreddit_details['id'][i], subreddit=subreddit_details['subreddit'][i],
                                        filter=['id','parent_id','permalink','author', 'title', 'subreddit','body','num_comments','score'], limit=30)  
  except:
    print("An exception occurred")
  
  df = pd.DataFrame(list(gen))
  comments_df = pd.concat([comments_df,df])


An exception occurred
An exception occurred
An exception occurred


In [ ]:
# total rows
print("Total rows: ", len(comments_df))

Total rows:  92654


In [ ]:
comments_df = comments_df.loc[comments_df['body'] != '[removed]']

In [ ]:
comments_df['full_link']

In [ ]:
# comments containing [removed] or [deleted] in body
removed(comments_df, 'com')
comments_df = comments_df.rename(columns={"permalink": "full_link"})

comments containing [removed] or [deleted] in body:  3.017


In [ ]:
comments_df

In [ ]:
# Add additional columns
topicsMap = {'explainlikeimfive': 'ExplainLikeImFive', 'Foodforthought': 'FoodForThought','changemyview': 'ChangeMyView', 'todayilearned': 'TodayILearned', 
             'worldnews' : 'Politics', 'worldpolitics' : 'Politics', 'government' : 'Politics', 
             'tentree' : 'Environment', 'nature' : 'Environment','gardening' : 'Environment', 
             'technology' :'Technology', 'tech' :'Technology','gaming' :'Technology', 
             'bodybeast' : 'Healthcare', 'BodyBeast': 'Healthcare', 'briskwalking' : 'Healthcare', 'crossfit' : 'Healthcare', 
             'education' :'Education','teaching' :'Education','UniversityofReddit' :'Education'}

comments_df['parent_body'] = 'blank'
comments_df['created_at'] = comments_df['created']

for i in range(len(comments_df)):
  date = comments_df['created_utc'].iloc[i]
  comments_df['created_at'].iloc[i] = pd.to_datetime(date, unit='s')

comments_df['is_submission'] = False

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
comments_df = comments_df[['id', 'subreddit', 'full_link', 'body', 'author', 'parent_id', 'parent_body', 'is_submission', 'created_at']]

In [ ]:
comments_df.groupby(['subreddit']).count()

In [ ]:
sum1 = 0
sum2 = 0
for i in range(len(comments_df)):
    parent = comments_df['parent_id'].iloc[i][:2]
    if parent == 't3':
        sum1 += 1
    else:
        sum2 += 1

print(sum1, sum2)

30490 59258


In [ ]:
comments_df.to_pickle("finalComments.pkl")

In [ ]:
comments_df.to_csv("finalComments.csv")

In [ ]:
# finalSubmissions_df = pd.read_pickle("finalSubmissions.pkl")  

In [ ]:
finalCombined = pd.concat([finalSubmissions, comments_df], axis=0)

In [ ]:
len(finalCombined)
removed(finalCombined, 'combined')

comments/submissions containing [removed] or [deleted] in body:  2.939


In [ ]:
finalCombined.to_pickle("finalCombined.pkl")

In [ ]:
finalCombined.to_csv("finalCombined.csv")

In [ ]:
len(finalCombined.loc[finalCombined['selftext'] == '[removed]']) + len(finalCombined.loc[finalCombined['selftext'] == '[deleted]']) + len(finalCombined.loc[finalCombined['body'] == '[removed]']) + len(finalCombined.loc[finalCombined['body'] == '[deleted]'])

3525

In [ ]:
3525/len(finalCombined) * 100


7.185372416323535

In [ ]:
# ParentBody

In [ ]:
test = pd.read_pickle("finalCombined_new.pkl")
# print(test.columns)


submissions = test.loc[test['is_submission'] == True]

comments = test.loc[test['is_submission'] == False]

print(len(submissions), len(comments), len(submissions) + len(comments), len(test))

3380 89748 93128 93128


In [ ]:
# Extracting t3 comments
from string import digits
for i in range(len(comments)):

  parent = comments['parent_id'].iloc[i][:2]
  parent_id = comments['parent_id'].iloc[i][3:]
  if parent == 't3':
    body = submissions['selftext'].loc[submissions['id'] == parent_id].values
    # body = body.lstrip(digits)
    comments['parent_body'].iloc[i] = body[0]
    # comments['parent_body'].iloc[i].str.lstrip(digits)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
# comments.to_csv("t3Comments.csv")
# submissions.to_csv("submissions.csv")

In [ ]:
t1_parent_ids = []
for i in range(len(comments)):
  parent = comments['parent_id'].iloc[i][:2]
  if parent == 't1':
    parent_id = comments['parent_id'].iloc[i][3:]
    t1_parent_ids.append(parent_id)

print(len(t1_parent_ids))

59258


In [ ]:
from logging import exception
final = pd.DataFrame()
for i in range(0, len(t1_parent_ids), 500):
  print(i, i+500)

  gen = api.search_comments(ids=t1_parent_ids[i:i+500],
                                     filter=['id','parent_id','permalink','author', 'title', 
                                                  'subreddit','body','num_comments','score','created_utc', 'created'],metadata=True, limit=1)  

  try:
    df = pd.DataFrame(list(gen))
  except:
    print(exception)
    # df = pd.DataFrame(list(gen)), columns=['id','parent_id','permalink','author', 'title', 
                                                  # 'subreddit','body','num_comments','score','created_utc', 'created','d_']
  df = df[['id', 'parent_id', 'body', 'created_utc']]
  final = pd.concat([final,df])



0 500
500 1000
1000 1500
1500 2000
2000 2500
2500 3000
3000 3500
3500 4000
4000 4500
4500 5000
5000 5500
5500 6000
<function exception at 0x7ff547ed6680>
6000 6500
6500 7000
7000 7500
7500 8000
8000 8500
8500 9000
9000 9500
9500 10000
10000 10500
10500 11000
11000 11500
11500 12000
12000 12500
12500 13000
13000 13500
13500 14000
14000 14500
14500 15000
15000 15500
15500 16000
16000 16500
16500 17000
<function exception at 0x7ff547ed6680>
17000 17500
17500 18000
<function exception at 0x7ff547ed6680>
18000 18500
18500 19000
19000 19500
19500 20000
20000 20500
<function exception at 0x7ff547ed6680>
20500 21000
<function exception at 0x7ff547ed6680>
21000 21500
21500 22000
<function exception at 0x7ff547ed6680>
22000 22500
22500 23000
23000 23500
23500 24000
24000 24500
<function exception at 0x7ff547ed6680>
24500 25000
<function exception at 0x7ff547ed6680>
25000 25500
25500 26000
26000 26500
<function exception at 0x7ff547ed6680>
26500 27000
27000 27500
27500 28000
28000 28500
28500 290

In [ ]:
len(final)

50410

In [ ]:
for i in range(len(comments)):
  parent = comments['parent_id'].iloc[i][:2]
  parent_id = comments['parent_id'].iloc[i][3:]
  if parent == 't1':
    if parent_id in final['id'].values:
      # print(parent_id)
      body = final['body'].loc[final['id'] == parent_id].values[0]
      comments['parent_body'].iloc[i] = body

In [ ]:
commentsCopy = pd.DataFrame()
commentsCopy = pd.concat([commentsCopy,comments])

In [ ]:
comments = commentsCopy

In [ ]:
comments['full_link'].iloc[29890]

'/r/nature/comments/qriif6/how_sarscov2_in_whitetailed_deer_could_alter_the/hkbbkwe/'

In [ ]:
import math
print(len(comments['full_link']))
print(len(comments['full_link'].loc[comments['full_link'].isna()]))

89748
6602


In [ ]:
for i in range(len(comments)):
  permalink = str(comments['full_link'].iloc[i])
  new_link = "https://www.reddit.com"+ permalink
  comments['full_link'].iloc[i] = new_link

In [ ]:
finalCombined_parentbody = pd.concat([submissions, comments])

In [ ]:
finalCombined_parentbody.loc[finalCombined_parentbody['is_submission'] == False]

In [ ]:
finalCombined_parentbody.to_pickle("finalCombined_parentbody.pkl")
finalCombined_parentbody.to_csv("finalCombined_parentbody.csv")